# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [27]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory

from azureml.core import Environment

from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.core import ScriptRunConfig
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()
experiment_name = 'amazon_review_prediction'

experiment=Experiment(ws, experiment_name)

ds = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 

In [13]:
found = False
dataset_name = "Amazon Reviews"
dataset_descriptions = "Set of Amazon products text reviews together with their rating value ranging from 1 to 5."

if dataset_name in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[dataset_name] 

if not found:
        dataset = TabularDatasetFactory.from_delimited_files(path="https://github.com/iolucas/AMLE_P3/raw/main/amazon_reviews.csv") 
        dataset = ds.register(workspace=ws, name=dataset_name, description=dataset_descriptions)

In [8]:
df = ds.to_pandas_dataframe()
print(df.shape)
df.head()

(28332, 2)


,review,rating
0,I order 3 of them and one of the item is bad q...,3
1,Bulk is always the less expensive way to go fo...,4
2,Well they are not Duracell but for the price i...,5
3,Seem to work as well as name brand batteries a...,5
4,These batteries are very long lasting the pric...,5


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [42]:
amlcompute_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS12_v2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first

AmlCompute is getting created. Consider calling wait_for_completion() first



In [43]:
compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

SucceededProvisioning operation finished, operation "Succeeded"
Succeeded.....................

KeyboardInterrupt: 

In [21]:
!python train.py

usage: train.py [-h] --data-dir DATA_DIR --output-dir OUTPUT_DIR
                --use-stopwords --use-binary-count --ngram-range NGRAM_RANGE
train.py: error: the following arguments are required: --data-dir, --output-dir, --use-stopwords, --use-binary-count, --ngram-range


In [76]:
from azureml.core import Environment

#sklearn_env2 = Environment.from_conda_specification(name='sklearn-env2', file_path='conda_dependencies.yml')
sklearn_env = Environment.get(ws, "AzureML-sklearn-0.24-ubuntu18.04-py37-cpu")

In [77]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    #"--C": uniform(0.0, 5.0),
    #"--max_iter": choice(100, 200, 300, 400)    
})

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      arguments=[
                          "--dataset-name", dataset_name,
                          "--output-dir", "outputs",
                          "--use-stopwords",
                          "--use-binary-count",
                          "--ngram-range", 1
                      ],
                      environment=sklearn_env
                     )

hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator, 
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=20,
    max_concurrent_runs=4
)

In [78]:
#TODO: Submit your experiment
hdr = experiment.submit(config=estimator)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [79]:
RunDetails(hdr).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service